<a href="https://colab.research.google.com/github/ricofelix/meu-projeto-ia-remoto/blob/main/TrabalhoML_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Projeto Final - Machine Learning I**

O projeto de finalização de módulo consiste em um fluxo de trabalho completo com Machine Learning. Para desenvolvê-lo, vocês devem escolher um dataset no Kaggle e desenvolver de ponta a ponta uma solução que utilize aprendizado de máquina.
Vamos ao passo a passo:

1.	Escolha um Dataset no Kaggle

i.	Um bom dataset para o projeto contêm pelo menos 1000 linhas

2.	Defina um problema de negócio a resolver com o Dataset (quero prever X, quero estimar Y)
3.	Faça o tratamento dos dados e as transformações que achar pertinentes usando o Pandas
4.	Determine a métrica de avaliação do seu modelo
5.	Elabore a solução de Machine Learning, usando pelo menos 2 estimadores (para comparar os resultados)
6.	Faça a otimização de hiperparametros para ambos os estimadores (usando o método mais adequado) e maximize a métrica de interesse.
7.	Compare os modelos e escolha o melhor
8.	Responda as perguntas:
i.	Meu modelo resolve adequadamente o problema proposto?
ii.	Meu modelo pode ser colocado em produção?


# **1 Escolha um Dataset no Kaggle**

i. Um bom dataset para o projeto contêm pelo menos 1000 linhas.

Vamos Trabalhar com o Dataset Bank_Personal_Loan_Modelling.csv

O dataset tem 5000 linhas e 14 colunas.

As principais colunas são: **Age**, **Experience**, **Income, Family, CCAvg (gasto médio no cartão), Education, Mortgage, Personal Loan** (empréstimo pessoal: alvo), e diversos tipos de contas e serviços bancários.





In [26]:
import pandas as pd
df = pd.read_csv('Bank_Personal_Loan_Modelling.csv')
df.head()
print(
    df.shape
)

(5000, 14)


# **2. Definição do Problema de Negócio**

Vamos prever se um cliente aceitará ou nao um emprestimo pessoal ( Personal Loan) usando as informações do perfil Bancário

Para isso vamos utilizar a **Classificação binária**



# 3.	**Faça o tratamento dos dados e as transformações**



Vamos remover as colunas **ID** e **ZIP Code** , pois não tem utilidade para previsão.

In [27]:
df_ml = df.drop(columns=['ID', 'ZIP Code'])
print(df_ml.isnull().sum())
print(df_ml.duplicated().sum())
df_clean = df_ml.drop_duplicates().reset_index(drop=True)
print(df_clean.describe().T)

Age                   0
Experience            0
Income                0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64
13
                     count       mean         std   min   25%   50%    75%  \
Age                 4987.0  45.347704   11.460838  23.0  35.0  45.0   55.0   
Experience          4987.0  20.114498   11.464902  -3.0  10.0  20.0   30.0   
Income              4987.0  73.824544   46.062728   8.0  39.0  64.0   98.0   
Family              4987.0   2.396631    1.147510   1.0   1.0   2.0    3.0   
CCAvg               4987.0   1.940630    1.748824   0.0   0.7   1.5    2.6   
Education           4987.0   1.879286    0.839667   1.0   1.0   2.0    3.0   
Mortgage            4987.0  56.646080  101.805341   0.0   0.0   0.0  101.0   
Personal Loan       4987.0   0.096250    0.294964   0.0   0.0   0.0    0

# **4.	Determine a métrica de avaliação do seu modelo**

Aqui vamos efetuar o treinamento e teste dos dados

In [28]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Separar variáveis preditoras e alvo
X = df_clean.drop('Personal Loan', axis=1)
y = df_clean['Personal Loan']

# Normalizar variáveis contínuas
num_cols = ['Age', 'Experience', 'Income', 'CCAvg', 'Mortgage']
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

# Separar conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Conferir os formatos
print('Treino:', X_train.shape)
print('Teste:', X_test.shape)
print('Total:', df_clean.shape[0])


Treino: (3740, 11)
Teste: (1247, 11)
Total: 4987


# **5.	Elabore a solução de Machine Learning, usando pelo menos 2 estimadores (para comparar os resultados)**

Vamos utilizar a Arvore de Decisão e Random Forest



In [29]:
# Importando algoritmos e métricas de avaliação
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
# Inicializando os modelos
dt_model = DecisionTreeClassifier(random_state=42)    # Árvore de Decisão
rf_model = RandomForestClassifier(random_state=42)    # Random Forest

# Treinando os modelos com os dados de Treino
dt_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)

# Usando os modelos para prever dados de teste
dt_pred = dt_model.predict(X_test)
rf_pred = rf_model.predict(X_test)

# Calculando a Acurácia: Porcentagem de acerto entre a previsao e a verdade
dt_acuracia = accuracy_score(y_test, dt_pred)
rf_acuracia = accuracy_score(y_test, rf_pred)

#Caulculando AUC-ROC
dt_auc= roc_auc_score(y_test, dt_model.predict_proba(X_test)[:, 1])
rf_auc= roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1])

# Mostrando resultados na tela
print(f'Acurácia Árvore de Decisão: {dt_acuracia:.4f} | AUC: {dt_auc:.4f}')
print(f'Acurácia Random Forest:    {rf_acuracia:.4f} | AUC: {rf_auc:.4f}')

Acurácia Árvore de Decisão: 0.9880 | AUC: 0.9785
Acurácia Random Forest:    0.9936 | AUC: 0.9981


# **6.	Faça a otimização de hiperparametros para ambos os estimadores (usando o método mais adequado) e maximize a métrica de interesse.**

In [30]:
from sklearn.model_selection import GridSearchCV

# Otimizando Árvore de Decisão
dt_params = {
    'max_depth': [3, 5, 7, 10, None],
    'criterion': ['gini', 'entropy']
}
dt_grid = GridSearchCV(DecisionTreeClassifier(random_state=42), dt_params,
                       scoring='roc_auc', cv=5, n_jobs=-1)
dt_grid.fit(X_train, y_train)

# Melhor configuração encontrada
print('Melhor Árvore de Decisão:', dt_grid.best_params_)

# Otimizando Random Forest
rf_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, None],
    'criterion': ['gini', 'entropy']
}
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params,
                       scoring='roc_auc', cv=5, n_jobs=-1)
rf_grid.fit(X_train, y_train)

# Melhor configuração encontrada
print('Melhor Random Forest:', rf_grid.best_params_)

Melhor Árvore de Decisão: {'criterion': 'entropy', 'max_depth': 3}
Melhor Random Forest: {'criterion': 'gini', 'max_depth': None, 'n_estimators': 100}


# **7.	Compare os modelos e escolha o melhor**

In [31]:
# Pegando os melhores modelos treinados pelo GridSearchCV
best_dt = dt_grid.best_estimator_
best_rf = rf_grid.best_estimator_

# Fazendo previsões com os modelos otimizados
dt_pred = best_dt.predict(X_test)
rf_pred = best_rf.predict(X_test)

# Calculando acurácia e AUC-ROC nos dados de teste
dt_accuracy = accuracy_score(y_test, dt_pred)
rf_accuracy = accuracy_score(y_test, rf_pred)

dt_auc = roc_auc_score(y_test, best_dt.predict_proba(X_test)[:, 1])
rf_auc = roc_auc_score(y_test, best_rf.predict_proba(X_test)[:, 1])

print(f"Árvore de Decisão - Accuracy: {dt_accuracy:.4f} | AUC: {dt_auc:.4f}")
print(f"Random Forest     - Accuracy: {rf_accuracy:.4f} | AUC: {rf_auc:.4f}")

# Comparando resultados lado a lado
if rf_auc > dt_auc:
    print("O modelo Random Forest teve o melhor desempenho e deve ser escolhido para produção.")
else:
    print("O modelo Árvore de Decisão teve o melhor desempenho e deve ser escolhido para produção.")

Árvore de Decisão - Accuracy: 0.9824 | AUC: 0.9946
Random Forest     - Accuracy: 0.9936 | AUC: 0.9981
O modelo Random Forest teve o melhor desempenho e deve ser escolhido para produção.


# **8.	Responda as perguntas:**

**i.	Meu modelo resolve adequadamente o problema proposto?**

Após realizar todas as etapas do projeto, consegui treinar e avaliar dois algoritmos clássicos de classificação para prever a aceitação do empréstimo pessoal. Um dos modelos, o Random Forest, apresentou um desempenho bastante satisfatório: a métrica AUC-ROC ficou elevada, indicando que o modelo separa bem quem provavelmente vai aceitar o empréstimo daqueles que não vão. A acurácia também ficou boa, mas como o importante no contexto bancário é identificar corretamente os clientes com maior chance de contratação, foquei mais na métrica AUC.
Portanto, sim, entendo que o modelo resolve o problema proposto e pode ser usado com segurança para apoiar decisões sobre oferta de empréstimos.

**ii.	Meu modelo pode ser colocado em produção?**

O pipeline está pronto para implantação. Os dados passaram pelas etapas necessárias para garantir qualidade (limpeza, normalização, retirada de duplicidades, validação do modelo em dados independentes). O modelo que teve melhor desempenho pode ser salvo e utilizado num sistema de produção, tanto via API quanto integrado a processos internos.
Naturalmente, recomendo sempre continuar monitorando os resultados, revisando o modelo conforme novas informações estiverem disponíveis e fazendo atualizações quando necessário para garantir que a performance se mantenha confiável.